# © Artur Czarnecki. All rights reserved.
# Intergrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.

## Test P1 — PlannerDynamicPipeline smoke: ScriptedPlanSource (SYNTHESIZE → FINALIZE)

Goal:
- Verify that `PlannerDynamicPipeline.run(state)` is correctly wired end-to-end:
  - runs SETUP steps,
  - enters DYNAMIC plan loop,
  - performs at least two iterations,
  - produces a non-empty final RuntimeAnswer.

Determinism:
- Planning is fully deterministic via `ScriptedPlanSource` injected through RuntimeConfig (`config.plan_source`).
- No fake LLM, no wrappers: only Intergrax components.

Assertions:
- Returned answer is non-empty.
- The loop performed 2 iterations (attempt None → 1 visible in planner trace).


In [ ]:
import sys, os

from intergrax.runtime.nexus.planning.engine_plan_models import EngineNextStep, PlanIntent
from intergrax.runtime.nexus.tracing.trace_models import TraceComponent
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

import pprint

from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry

from intergrax.runtime.nexus.config import RuntimeConfig
from intergrax.runtime.nexus.engine.runtime_context import RuntimeContext
from intergrax.runtime.nexus.engine.runtime_state import RuntimeState
from intergrax.runtime.nexus.responses.response_schema import RuntimeRequest

from intergrax.runtime.nexus.pipelines.planner_dynamic_pipeline import PlannerDynamicPipeline

from intergrax.runtime.nexus.planning.plan_sources import PlanSpec, ScriptedPlanSource
from intergrax.runtime.nexus.planning.plan_loop_models import PlanLoopPolicy
from intergrax.runtime.nexus.planning.step_planner import StepPlannerConfig
from intergrax.runtime.nexus.planning.step_executor_models import StepExecutorConfig

from intergrax.runtime.nexus.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.nexus.session.session_manager import SessionManager


# 1) Deterministic plans (attempt=0 -> attempt=1)
plan_0_synthesize = PlanSpec(
    version="1",
    intent=PlanIntent.GENERIC,
    next_step=EngineNextStep.SYNTHESIZE,
    reasoning_summary="Produce a draft first (pipeline smoke).",
    ask_clarifying_question=False,
    clarifying_question=None,
    use_websearch=False,
    use_user_longterm_memory=False,
    use_rag=False,
    use_tools=False,
)

plan_1_finalize = PlanSpec(
    version="1",
    intent=PlanIntent.GENERIC,
    next_step=EngineNextStep.FINALIZE,
    reasoning_summary="Finalize using the produced draft (pipeline smoke).",
    ask_clarifying_question=False,
    clarifying_question=None,
    use_websearch=False,
    use_user_longterm_memory=False,
    use_rag=False,
    use_tools=False,
)

scripted_source = ScriptedPlanSource([plan_0_synthesize, plan_1_finalize])

# 2) Runtime config
llm_adapter = LLMAdapterRegistry.create(LLMProvider.OLLAMA)

config = RuntimeConfig(
    llm_adapter=llm_adapter,
    enable_rag=False,
    enable_websearch=False,
    tools_mode="off",
    step_planner_cfg=StepPlannerConfig(),
    step_executor_cfg=StepExecutorConfig(),
    plan_loop_policy=PlanLoopPolicy(max_replans=3, max_same_plan_repeats=2),
    plan_source=scripted_source,  # <--- requires the small RuntimeConfig addition
)

# 3) Context + state
session_manager = SessionManager(storage=InMemorySessionStorage())

ctx = RuntimeContext.build(
    config=config,
    session_manager=session_manager,
)

req = RuntimeRequest(
    session_id="dyn-pipeline-test-session-p1",
    user_id="dyn-pipeline-test-user",
    message="P1: dynamic pipeline deterministic smoke test (SYNTHESIZE -> FINALIZE).",
    attachments=[],
)

state = RuntimeState(
    context=ctx,
    request=req,
    run_id="planloop-e2e-run",
)

state.cap_websearch_available = True
state.cap_tools_available = True
state.cap_rag_available = True
state.cap_user_ltm_available = True

# 4) Run pipeline
pipeline = PlannerDynamicPipeline()
answer = await pipeline.run(state)

print("Answer:", (answer.answer or "")[:250])
print("Route.strategy:", answer.route.strategy)
print("Route.extra:", answer.route.extra)

# 5) Assertions
assert answer.answer and answer.answer.strip(), "Expected non-empty final answer."
assert state.runtime_answer is answer, "state.runtime_answer must be set to returned answer."

events = state.trace_events

planner_iters = [
    e for e in events
    if e.component == TraceComponent.PLANNER
    and e.step == "plan_loop_dynamic"
    and e.message == "Planning iteration started."
]

print("planner_iterations_count:", len(planner_iters))
for i, e in enumerate(planner_iters):
    replan_attempt = e.data.replan_attempt if e.data is not None else None
    print(f"iter[{i}] replan_attempt:", replan_attempt)

assert len(planner_iters) == 2, f"Expected exactly 2 iterations, got {len(planner_iters)}"
assert (planner_iters[0].data.replan_attempt if planner_iters[0].data is not None else None) is None
assert (planner_iters[1].data.replan_attempt if planner_iters[1].data is not None else None) == 1

print("P1 OK")



Answer: It seems like you're referring to a software development process, specifically a test phase for a project. Here's a breakdown of the terms and what they imply:

- **Dynamic pipeline**: This term is often used in continuous integration/continuous depl
Route.strategy: llm_only
Route.extra: {'used_attachments_context': False, 'attachments_chunks': 0}
planner_iterations_count: 2
iter[0] replan_attempt: None
iter[1] replan_attempt: 1
P1 OK


## Test P2 — PlannerDynamicPipeline smoke: Scripted CLARIFY → HITL

Goal:
- Verify that `PlannerDynamicPipeline.run(state)` returns a HITL clarifying question when the planner decides to clarify.
- Planning is deterministic via ScriptedPlanSource (attempt=0).
- No extra components, no fake LLM.

Assertions:
- answer.route.strategy == "hitl_clarify"
- answer.answer equals the scripted clarifying question
- planner iterations count == 1


In [ ]:
import sys, os

from intergrax.runtime.nexus.tracing.trace_models import TraceComponent
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry

from intergrax.runtime.nexus.config import RuntimeConfig
from intergrax.runtime.nexus.engine.runtime_context import RuntimeContext
from intergrax.runtime.nexus.engine.runtime_state import RuntimeState
from intergrax.runtime.nexus.responses.response_schema import RuntimeRequest

from intergrax.runtime.nexus.pipelines.planner_dynamic_pipeline import PlannerDynamicPipeline

from intergrax.runtime.nexus.planning.plan_sources import PlanSpec, ScriptedPlanSource
from intergrax.runtime.nexus.planning.plan_loop_models import PlanLoopPolicy
from intergrax.runtime.nexus.planning.step_planner import StepPlannerConfig
from intergrax.runtime.nexus.planning.step_executor_models import StepExecutorConfig

from intergrax.runtime.nexus.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.nexus.session.session_manager import SessionManager


CLAR_Q = "Please clarify which input data sources should be used for the dynamic pipeline test (RAG, websearch, tools, or none)."

plan_clarify = PlanSpec(
    version="1",
    intent=PlanIntent.CLARIFY,
    next_step=EngineNextStep.CLARIFY,
    reasoning_summary="Need user clarification before proceeding.",
    ask_clarifying_question=True,
    clarifying_question=CLAR_Q,
    use_websearch=False,
    use_user_longterm_memory=False,
    use_rag=False,
    use_tools=False,
)


scripted_source = ScriptedPlanSource([plan_clarify])

llm_adapter = LLMAdapterRegistry.create(LLMProvider.OLLAMA)

config = RuntimeConfig(
    llm_adapter=llm_adapter,
    enable_rag=False,
    enable_websearch=False,
    tools_mode="off",
    step_planner_cfg=StepPlannerConfig(),
    step_executor_cfg=StepExecutorConfig(),
    plan_loop_policy=PlanLoopPolicy(max_replans=3, max_same_plan_repeats=2),
    plan_source=scripted_source,
    # planner_prompt_config intentionally None for scripted planning
)

session_manager = SessionManager(storage=InMemorySessionStorage())
ctx = RuntimeContext.build(config=config, session_manager=session_manager)

req = RuntimeRequest(
    session_id="dyn-pipeline-test-session-p2",
    user_id="dyn-pipeline-test-user",
    message="P2: dynamic pipeline should return HITL clarify.",
    attachments=[],
)

state = RuntimeState(
    context=ctx,
    request=req,
    run_id="dyn-pipeline-p2-run",
)

state.cap_websearch_available = True
state.cap_tools_available = True
state.cap_rag_available = True
state.cap_user_ltm_available = True

pipeline = PlannerDynamicPipeline()
answer = await pipeline.run(state)

print("Answer:", answer.answer)
print("Route.strategy:", answer.route.strategy)
print("Route.extra:", answer.route.extra)

assert answer.route.strategy == "hitl_clarify", f"Expected hitl_clarify, got {answer.route.strategy!r}"
assert answer.answer == CLAR_Q, "Expected answer to equal scripted clarifying question."

events = state.trace_events

planner_iters = [
    e for e in events
    if e.component == TraceComponent.PLANNER
    and e.step == "plan_loop_dynamic"
    and e.message == "Planning iteration started."
]

print("planner_iterations_count:", len(planner_iters))
assert len(planner_iters) == 1, f"Expected exactly 1 iteration, got {len(planner_iters)}"

print("P2 OK")


Answer: Please clarify which input data sources should be used for the dynamic pipeline test (RAG, websearch, tools, or none).
Route.strategy: hitl_clarify
Route.extra: {'stop_reason': 'needs_user_input', 'origin_step_id': <StepId.CLARIFY: 'clarify'>, 'context_key': 'clarify.user_input', 'must_answer_to_continue': True}
planner_iterations_count: 1
P2 OK
